In [3]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path("/home/gabriella/mase/machop")
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
# import torchvision
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# import torch_tensorrt

# from torch.utils.tensorboard import SummaryWriter

# import pytorch_quantization
# from pytorch_quantization import nn as quant_nn
# from pytorch_quantization import quant_modules
# from pytorch_quantization.tensor_quant import QuantDescriptor
# from pytorch_quantization import calib
# from tqdm import tqdm

# print(pytorch_quantization.__version__)

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.tools import get_cf_args, get_dummy_input, load_config
from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    quantize_tensorrt_transform_pass,
    test_quantize_tensorrt_transform_pass,
    fake_quantize_transform_pass,
    graph_calibration_pass,
    evaluate_fake_quantize_pass,
    fake_quantize_to_trt_pass
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model, get_tokenizer

set_logging_verbosity("info")

TypeError: unsupported operand type(s) for |: 'type' and 'type'

In [2]:
batch_size = 8
model_name = "vgg7"
dataset_name = "cifar10"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


# load the VGG model
CHECKPOINT_PATH = "/home/gabriella/mase/machop/test-accu-0.9332.ckpt"
# CHECKPOINT_PATH = "/home/qizhu/Desktop/Work/mase/mase_output/opt125.ckpt"

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

NameError: name 'MaseDataModule' is not defined

In [ ]:
#transfer the model into mase graph
mg = MaseGraph(model=model)
ori_mg = MaseGraph(model=model)

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
# _ = model(**dummy_in)

#add all the parameters to masegraph
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

### 

### Quantization pass based on pytorch-quantization

To achieve QAT, we could use fake quantization to simulate the quantization process. We could use the `FakeQuantize` argument to select it.

Here's an example of pass_args, we use name to specify the layer we want to quantize, achieving layer-wise PTQ, and we could select the specific calibrate method and precision.

In [ ]:
## fake quantize the graph
pass_args = {
    "by": "name",
    "default": {"config": {"name": None}},
    "feature_layers_0": {
        "config": {
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "max",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "histogram",
                "quantize_axis": None,
            },
        }
    },    
    "classifier_0": {
        "config": {
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_1": {
        "config": {
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_2": {
        "config": {
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_3": {
        "config": {
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
}

mg = fake_quantize_transform_pass(mg, pass_args)


### Calibration

After quantization the model, we need to calibrate the model and get amax to each layer. We can use the following pass_args to contral the calibration process, including the method, the number of samples, and the calibration dataset.

In [ ]:
#the pass of calibration for tenssor_quantization
pass_args_calibrate = {
    "calibrator": "percentile",
    "percentiles": [99],
    "data_module": data_module,
    "num_batches": 100,
}

graph_calibration_pass(mg,  pass_args_calibrate)

### Evaluate


In [ ]:
pass_args_eval = {
    "data_module": data_module,
}

mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

### Model export

We first need to export the model to ONNX format, then to tensorrt engine.

In [ ]:
#using tenssor quantization
pass_args = {
    "onnxFile": "onnx_test.onnx",
    "engineFile": "engine_test.plan",
    "dataloader": data_module.test_dataloader,
}
mg = fake_quantize_to_trt_pass(mg, pass_args)
